Here, in line-up prediction we cannot use a whole year as the test set. Since many changes have been occured (data shift), we have many new players this year and many retired.

In [1]:
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
#from sklearn.preprocessing import OneHotEncoder
"""from google.colab import drive


drive.mount('/content/drive')
ds_path = '/content/drive/MyDrive/Data_Mining_Project/Datasets/'"""

ds_path = 'Datasets/'
assert os.path.exists(ds_path)

home_players_columns = ['home_0','home_1','home_2','home_3','home_4']
away_players_columns = ['away_0','away_1','away_2','away_3','away_4']
features = home_players_columns + away_players_columns

df = pd.DataFrame()
# Load data from matchups-2007.csv to matchups-2012.csv and append them to df
for i in range(2007, 2013):
    df1 = pd.read_csv(ds_path + "matchups-" + str(i) + ".csv")[features]
    print(f"size of the matchup {i} is {len(df1)}")
    df = pd.concat([df, df1])
df.reset_index(drop=True, inplace=True)
print(f"len of final df: {len(df)}")
print("first print \n" + str(df))
print(df.shape)


raw_df = df.copy()

size of the matchup 2007 is 27500
size of the matchup 2008 is 26593
size of the matchup 2009 is 26407
size of the matchup 2010 is 26344
size of the matchup 2011 is 26447
size of the matchup 2012 is 21241
len of final df: 154532
first print 
                 home_0            home_1            home_2            home_3  \
0          Andrew Bynum        Lamar Odom       Luke Walton     Sasha Vujacic   
1          Andrew Bynum        Lamar Odom       Luke Walton     Sasha Vujacic   
2            Lamar Odom       Luke Walton     Maurice Evans      Ronny Turiaf   
3            Lamar Odom       Luke Walton     Maurice Evans      Ronny Turiaf   
4           Luke Walton     Maurice Evans      Ronny Turiaf      Smush Parker   
...                 ...               ...               ...               ...   
154527  Bismack Biyombo  Gerald Henderson      Jamario Moon      Kemba Walker   
154528  Bismack Biyombo  Gerald Henderson      Jamario Moon      Kemba Walker   
154529  Bismack Biyombo     By

In [2]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model

# Label encoding for teams and players
player_le = LabelEncoder()

# Encoding players
# Flatten the DataFrame to encode all player positions together
player_columns = [f'home_{i}' for i in range(5)] + [f'away_{i}' for i in range(5)]
all_players = raw_df[player_columns].values.flatten()

# Fit the LabelEncoder on all player names
player_le.fit(all_players)

# Transform each player column in the DataFrame
for col in player_columns:
    df[col] = player_le.transform(raw_df[col])

print(df)

        home_0  home_1  home_2  home_3  home_4  away_0  away_1  away_2  \
0           41     482     511     692     715      81     474     634   
1           41     482     511     692     715      26     494     634   
2          482     511     554     673     715      26     494     634   
3          482     511     554     673     715      81     346     474   
4          511     554     673     715     785      81     346     474   
...        ...     ...     ...     ...     ...     ...     ...     ...   
154527      72     290     340     453     776      13     104     303   
154528      72     290     340     453     776      13     303     305   
154529      72     100     216     290     453      13     303     305   
154530      72     216     290     453     776      13     303     305   
154531      72     216     290     453     776      13     104     303   

        away_3  away_4  
0          710     729  
1          710     729  
2          710     729  
3          

In [3]:
# OTHER
import pandas as pd
import numpy as np

# Assuming df is your DataFrame with the player columns
# Example columns: ['home_0', 'home_1', ..., 'away_4']


def preprocess_data_expanded(df1):
    # Initialize a list to hold new rows
    new_rows = []
    
    for index, row in df1.iterrows():
        # Get all player names for the row
        players = row[features].tolist()
        
        # Iterate over all players, selecting each one as the target once
        for target_player in players:
            # Use the remaining players as the context
            context_players = [player for player in players if player != target_player]
            
            # Create a new row with this player as the target
            new_row = row.copy()
            new_row['target_player_name'] = target_player
            new_row['player_names'] = context_players
            
            # Append the new row to the list
            new_rows.append(new_row)
    
    # Create a new DataFrame from the list of new rows
    new_df = pd.DataFrame(new_rows).reset_index(drop=True)
    
    # Drop the original player columns
    new_df = new_df.drop(features, axis=1)
    return new_df


# Apply preprocessing
df_preprocessed = preprocess_data_expanded(df)

In [4]:
# Flatten the DataFrame slice into a single array of player names
player_names_array = df[features].values.flatten()

# Find the unique player names in the array
unique_player_names = np.unique(player_names_array)
print(len(unique_player_names))

812


In [5]:
# OTHER
print(df_preprocessed)
print(df_preprocessed.loc[1,'player_names'])
print(len(unique_player_names))

         target_player_name                                  player_names
0                        41  [482, 511, 692, 715, 81, 474, 634, 710, 729]
1                       482   [41, 511, 692, 715, 81, 474, 634, 710, 729]
2                       511   [41, 482, 692, 715, 81, 474, 634, 710, 729]
3                       692   [41, 482, 511, 715, 81, 474, 634, 710, 729]
4                       715   [41, 482, 511, 692, 81, 474, 634, 710, 729]
...                     ...                                           ...
1545315                  13  [72, 216, 290, 453, 776, 104, 303, 305, 527]
1545316                 104   [72, 216, 290, 453, 776, 13, 303, 305, 527]
1545317                 303   [72, 216, 290, 453, 776, 13, 104, 305, 527]
1545318                 305   [72, 216, 290, 453, 776, 13, 104, 303, 527]
1545319                 527   [72, 216, 290, 453, 776, 13, 104, 303, 305]

[1545320 rows x 2 columns]
[41, 511, 692, 715, 81, 474, 634, 710, 729]
812


In [6]:
# data split
X_train_bu, X_test = train_test_split(df_preprocessed, test_size=0.2, random_state=52)
print(X_train_bu)
print(X_test)

         target_player_name                                   player_names
609944                  362   [55, 232, 271, 349, 178, 544, 643, 737, 770]
787005                   33   [306, 524, 569, 600, 630, 34, 689, 746, 799]
1092050                  26  [176, 486, 647, 801, 106, 125, 245, 401, 493]
872852                  199       [12, 65, 444, 529, 27, 36, 55, 356, 527]
612864                  675    [33, 503, 545, 648, 90, 356, 426, 643, 759]
...                     ...                                            ...
1380503                 665   [315, 366, 482, 710, 17, 101, 219, 250, 263]
1105421                 121    [1, 152, 417, 505, 229, 365, 392, 534, 799]
1026827                 272   [454, 459, 623, 635, 646, 40, 119, 504, 509]
566940                   38    [93, 471, 476, 622, 49, 102, 338, 658, 673]
1530869                 687   [63, 246, 329, 683, 691, 276, 396, 490, 597]

[1236256 rows x 2 columns]
         target_player_name                                   player_nam

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization

def select_model(num):
    global model
    if num == 1:
        player_input_dim = len(player_le.classes_)

        player_input = Input(shape=(9,), dtype='int32', name='player_input')
        player_embedding = Embedding(input_dim=player_input_dim + 1, output_dim=64)(player_input)
        player_embedding_pooled = GlobalAveragePooling1D()(player_embedding)
        
        dense_layer = Dense(256, activation='relu')(player_embedding_pooled)
        dense_layer = Dense(256, activation='relu')(dense_layer)
        output_layer = Dense(player_input_dim, activation='softmax')(dense_layer)
        
        
    elif num == 2:
        player_input_dim = len(player_le.classes_)

        player_input = Input(shape=(9,), dtype='int32', name='player_input')
        player_embedding = Embedding(input_dim=player_input_dim + 1, output_dim=64)(player_input)
        player_embedding_pooled = GlobalAveragePooling1D()(player_embedding)
        
        dense_layer = Dense(512, activation='relu')(player_embedding_pooled)
        dense_layer = Dense(256, activation='relu')(dense_layer)
        output_layer = Dense(player_input_dim, activation='softmax')(dense_layer)
        
        

    elif num == 3:
        player_input_dim = len(player_le.classes_)

        player_input = Input(shape=(9,), dtype='int32', name='player_input')
        player_embedding = Embedding(input_dim=player_input_dim + 1, output_dim=64)(player_input)
        player_embedding_pooled = GlobalAveragePooling1D()(player_embedding)
        
        dense_layer = Dense(128, activation='relu')(player_embedding_pooled)
        dense_layer = Dense(128, activation='relu')(dense_layer)
        dense_layer = Dense(128, activation='relu')(dense_layer)
        output_layer = Dense(player_input_dim, activation='softmax')(dense_layer)
        
        
    #elif num == 2:
    model = Model(inputs=player_input, outputs=output_layer)    

In [8]:
epochs = 20

def lr_schedule(epoch):
    """Learning rate schedule function."""
    initial_lr = 0.001
    if epoch < epochs / 4:
        return initial_lr
    else:
        return initial_lr * np.exp(-0.1 * epoch)

In [9]:
#only players
from tensorflow.keras.layers import Input, Embedding, concatenate, Dropout, Dense, GlobalAveragePooling1D, Reshape, Flatten
from tensorflow.keras.callbacks import TensorBoard, LearningRateScheduler
import shutil
from numpy.random import seed
from keras.utils import pad_sequences

res_path = 'Results2/'
def train_NN(mod_num):
    accuracy_list = []
    
    for a_seed in range(100, 111):
        np.random.seed(a_seed)
        seed(a_seed)
        select_model(mod_num)
        
        X_train, X_val = train_test_split(X_train_bu, test_size=0.2, random_state=a_seed)
        
        lr_scheduler = LearningRateScheduler(lr_schedule)
        
        # Compile the model
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        
        player_names_padded_train = pad_sequences(X_train['player_names'], maxlen=9, padding='post', value=0)
        target_player_names_train = tf.cast(np.array(X_train['target_player_name'].values).astype(np.float32), dtype=tf.int32)
        player_names_padded_val = pad_sequences(X_val['player_names'], maxlen=9, padding='post', value=0)
        target_player_names_val = tf.cast(np.array(X_val['target_player_name'].values).astype(np.float32), dtype=tf.int32)
    
        # Train the model
        history = model.fit(player_names_padded_train, target_player_names_train, epochs=epochs, 
                            validation_data=(player_names_padded_val, target_player_names_val), verbose=False,
                            callbacks=[lr_scheduler])

        loss, accuracy = model.evaluate(player_names_padded_val, target_player_names_val)
        accuracy_list.append(accuracy)


    
    
    # Retrain and Evaluate the model
    # Clear log directory if it exists
    log_dir = "logs23_ModelNum" + str(mod_num) + "/fit/"
    if os.path.exists(log_dir):
        shutil.rmtree(log_dir)
    os.makedirs(log_dir)
    sm_checkpoint = tf.keras.callbacks.ModelCheckpoint(res_path + 'checkpoints/best_model_' + str(mod_num) + '.h5', 
                                							monitor='val_accuracy', verbose=0, 
                                							save_best_only=True, mode='max')
    # Define TensorBoard callback
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
    select_model(mod_num)
    lr_scheduler = LearningRateScheduler(lr_schedule)
    
    model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

    model.summary()
    
    player_names_padded = pad_sequences(X_train_bu['player_names'], maxlen=9, padding='post', value=0)
    target_player_names = tf.cast(np.array(X_train_bu['target_player_name'].values).astype(np.float32), dtype=tf.int32)
    test_player_names_padded = pad_sequences(X_test['player_names'], maxlen=9, padding='post', value=0)
    test_target_player_names = tf.cast(np.array(X_test['target_player_name'].values).astype(np.float32), dtype=tf.int32)
    
    history = model.fit(player_names_padded, target_player_names, epochs=epochs, validation_data=(test_player_names_padded, test_target_player_names), 
                        verbose=False,
                        callbacks=[tensorboard_callback, lr_scheduler, sm_checkpoint])

    loss, accuracy = model.evaluate(test_player_names_padded, test_target_player_names)
    
    print(f'Test Loss: {loss:.4f}')
    print(f'Test Accuracy: {accuracy:.4f}')
    
    print()
    print("From Cross_validation:")
    print(f"Accuracy: {np.mean(accuracy_list):.4f}, std: {np.std(accuracy_list):.4f}")
    
    

In [21]:
train_NN(1)

7727/7727 [==============================] - 11s 1ms/step - loss: 2.0549 - accuracy: 0.4031
Model: "model_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 player_input (InputLayer)   [(None, 9)]               0         
                                                                 
 embedding_24 (Embedding)    (None, 9, 64)             52032     
                                                                 
 global_average_pooling1d_2  (None, 64)                0         
 4 (GlobalAveragePooling1D)                                      
                                                                 
 dense_72 (Dense)            (None, 256)               16640     
                                                                 
 dense_73 (Dense)            (None, 256)               65792     
                                                                 
 dense_74 (Dense)            (No

C:\Python-3.11.4\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


9659/9659 [==============================] - 13s 1ms/step - loss: 2.0164 - accuracy: 0.4051
Test Loss: 2.0164
Test Accuracy: 0.4051

From Cross_validation:
Accuracy: 0.3939, std: 0.0039


In [22]:
train_NN(2)

7727/7727 [==============================] - 10s 1ms/step - loss: 1.9993 - accuracy: 0.4099
Model: "model_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 player_input (InputLayer)   [(None, 9)]               0         
                                                                 
 embedding_36 (Embedding)    (None, 9, 64)             52032     
                                                                 
 global_average_pooling1d_3  (None, 64)                0         
 6 (GlobalAveragePooling1D)                                      
                                                                 
 dense_108 (Dense)           (None, 512)               33280     
                                                                 
 dense_109 (Dense)           (None, 256)               131328    
                                                                 
 dense_110 (Dense)           (No

In [10]:
train_NN(3)

7727/7727 [==============================] - 8s 1ms/step - loss: 2.2350 - accuracy: 0.3427
Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 player_input (InputLayer)   [(None, 9)]               0         
                                                                 
 embedding_11 (Embedding)    (None, 9, 64)             52032     
                                                                 
 global_average_pooling1d_1  (None, 64)                0         
 1 (GlobalAveragePooling1D)                                      
                                                                 
 dense_44 (Dense)            (None, 128)               8320      
                                                                 
 dense_45 (Dense)            (None, 128)               16512     
                                                                 
 dense_46 (Dense)            (Non

C:\Python-3.11.4\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


9659/9659 [==============================] - 11s 1ms/step - loss: 2.1917 - accuracy: 0.3451
Test Loss: 2.1917
Test Accuracy: 0.3451

From Cross_validation:
Accuracy: 0.3337, std: 0.0069
